In [1]:
import time

from model import Model, Optimizer, WavenetData

from IPython.display import Audio
from matplotlib import pyplot as plt
from matplotlib import pylab as pl
from IPython import display
import torch
import numpy as np
#%matplotlib inline
%matplotlib notebook

In [59]:
model = Model(num_time_samples=10000, 
              num_blocks=1, 
              num_layers=10, 
              num_hidden=32, 
              num_classes=64)
torch.save(model, 'untrained_model')
print('scope: ', model.scope)

optimizer = Optimizer(model, 
                      stop_threshold=0.5)

b0-l0
current scope:  1
b0-l1
current scope:  3
b0-l2
current scope:  7
b0-l3
current scope:  15
b0-l4
current scope:  31
b0-l5
current scope:  63
b0-l6
current scope:  127
b0-l7
current scope:  255
b0-l8
current scope:  511
scope:  1535
scope:  1535


In [1]:
data = WavenetData('voice.wav',
                   input_length=model.scope,
                   target_length=model.last_block_scope,
                   num_classes=model.num_classes)
start_tensor = data.get_minibatch([12345])[0].squeeze()
plt.ion()
plt.plot(start_tensor.numpy())
plt.ioff()

NameError: name 'WavenetData' is not defined

In [1]:
fig = plt.figure()
ax = fig.add_subplot(111)
plt.ion()

fig.show()
fig.canvas.draw()

def hook(losses):
    ax.clear()
    ax.plot(losses)
    fig.canvas.draw()
    #print(losses[-1])
    
optimizer.hook = hook

print('start training...')
tic = time.time()
optimizer.train(data)
toc = time.time()
print('Training took {} seconds.'.format(toc-tic))

NameError: name 'plt' is not defined

In [1]:
print('generate...')
tic = time.time()
generated = model.generate(start_data=start_tensor, num_generate=model.scope)
#generated = model.fast_generate(40000, first_sample=0.1)
toc = time.time()
print('Generating took {} seconds.'.format(toc-tic))

generate...


NameError: name 'time' is not defined

In [1]:
fig = plt.figure()
plt.plot(generated.data[-model.scope:].numpy())

NameError: name 'plt' is not defined